<a href="https://colab.research.google.com/github/sondoanletrung-arch/Code_optimization/blob/main/R_code/Origin_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
library(Seurat)
library(dplyr)
library(SingleR)
library(celldex)
library(SingleCellExperiment)


data_dir <- "C:/Users/Administrator/Downloads/GSE282701_RAW-20251128T111238Z-1-001"
# Đường dẫn lưu các file kết quả trung gian
work_dir <- "C:/gse"

# Tạo seurat object
sample_folders <- list.dirs(data_dir, full.names = FALSE, recursive = FALSE)
seurat_list <- list()

for (folder in sample_folders) {
  message(paste("Đang đọc mẫu:", folder))
  data_counts <- Read10X(data.dir = file.path(data_dir, folder))

  obj <- CreateSeuratObject(counts = data_counts, project = folder, min.cells = 3, min.features = 200)

  # Thêm thông tin loại mẫu (Tumor/Paracancerous)
  obj$TissueType <- ifelse(grepl("T$", folder), "Tumor", "Paracancerous")
  seurat_list[[folder]] <- obj
}

# Gộp thành 1 object lớn
sr <- merge(x = seurat_list[[1]], y = seurat_list[2:length(seurat_list)],
                         add.cell.ids = names(seurat_list), project = "GSE282701_Liver")

# Seurat v5 cần lệnh này để gộp các layer lại
sr <- JoinLayers(sr)

# Lưu raw count cho inferCNV
sr[["RNA"]]$raw_count <- sr[["RNA"]]$counts

#rm(seurat_list); gc() # Dọn rác

#QC, normalize, PCA, UMAP
sr[["percent.mt"]] <- PercentageFeatureSet(sr, pattern = "^MT-")


sr <- subset(sr, subset = nFeature_RNA > 200 & nFeature_RNA < 6000 & percent.mt < 15)
sr <- NormalizeData(sr)

sr <- FindVariableFeatures(sr, selection.method = "vst", nfeatures = 2000)

all.genes <- rownames(sr)
sr <- ScaleData(sr, features = all.genes)

sr <- RunPCA(sr, features = VariableFeatures(object = sr))

sr <- FindNeighbors(sr, dims = 1:20)
sr <- FindClusters(sr, resolution = 0.5)
sr <- RunUMAP(sr, dims = 1:20)

# Dùng singleR để annotate
ref <- HumanPrimaryCellAtlasData()

# Chuyển sang SingleCellExperiment để chạy SingleR
sce <- as.SingleCellExperiment(sr)
pred.main <- SingleR(test = sce, ref = ref, labels = ref$label.main)

# Gán kết quả ngược lại vào Seurat
sr$SingleR.labels <- pred.main$labels

# Chạy lại UMAP để kiểm tra kết quả annotation
pl <- DimPlot(sr, reduction = "umap", group.by = "SingleR.labels", label = TRUE, repel = TRUE) +
      ggtitle("SingleR Annotations") + NoLegend()

pl
# Lưu object
saveRDS(sr, file = file.path(work_dir, "GSE282701_Seurat_Annotated.rds"))

In [ ]:
library(Seurat)

# Load lại nếu lỡ tắt máy (nếu đang chạy liên tục thì bỏ qua dòng này)
# sr <- readRDS("C:/gse/GSE282701_Seurat_Annotated.rds")
# inferCNV cần 3 file, annotation, raw count, gene ordering file
annot_df <- data.frame(
  cell_id = colnames(sr),
  cell_type = sr$SingleR.labels
)

write.table(annot_df, file = "C:/gse/infercnv_annotations.txt",
            sep = "\t", quote = FALSE, row.names = FALSE, col.names = FALSE)

# Lấy ma trận raw count
raw_matrix <- GetAssayData(sr, layer = "raw_count")

# Dọn RAM
rm(sr, annot_df, sce, pred.main)
gc()

In [ ]:
library(infercnv)

#tạo object infercnv
infercnv_obj <- CreateInfercnvObject(
  raw_counts_matrix = raw_matrix,
  annotations_file = "C:/gse/infercnv_annotations.txt",
  delim = "\t",
  gene_order_file = "C:/gse/gene_ordering_file.txt",
  ref_group_names = c("T_cells")
)


#chạy inferCNV nhưng không có HMM, tránh tràn RAM
out_dir <- "C:/gse/infercnv_output_Final"

infercnv_obj = infercnv::run(
  infercnv_obj,
  cutoff = 0.1,
  out_dir = out_dir,
  cluster_by_groups = T,
  denoise = T,
  HMM = F,
  num_threads = 4
)

In [ ]:
# Chạy hết code
infercnv_obj = infercnv::run(infercnv_obj,
                             cutoff=1,
                             out_dir="C:/gse/infercnv_output_HMM",
                             cluster_by_groups=T,
                             denoise=T,
                             HMM=T, #
                             num_threads = 4
                             )

In [ ]:
library(infercnv)

#Lad lại kết quả đã chạy và lưu trên drive
infercnv_obj <- readRDS("C:/gse/infercnv_output_noHMM/15_tumor_subclusters.leiden.infercnv_obj")


plot_cnv(
    infercnv_obj,
    out_dir = "C:/gse/infercnv_output_noHMM/Heatmap",
    cluster_by_groups = T,
    title = "InferCNV Heatmap",
    dynamic_resize = 0,
    output_format = "png",
    png_res = 300,
)